## Stochastic Optimization for Vehicle Routing Problem with PYOMO

### Import Necessary Libraries

In [1]:
!pip install pyomo
!sudo apt-get install glpk-utils

from pyomo.environ import *
from pyomo import environ as pe
# from coopr.pyomo import *
#from coopr.opt.base import solver
from pyomo.opt import *

from google.colab import drive
from google.colab import files

import scipy
import numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 28.3 MB/s 
     |████████████████████████████████| 49 kB 3.2 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 5 not upgraded.
Need to get 692 kB of archives.
After this operation, 1,664 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.c

In [3]:
# Mount Google Drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import Data File
!cp drive/MyDrive/Colab_Data/svrp_data.dat "/content/"

### Abstract Model

In [5]:
# AbstractModel is where data values are supplied in a data file

model = AbstractModel()

In [6]:
# Define sets

model.I = Set() #node
model.J = Set()
model.S = Set() #source node
model.D = Set() #demand node

In [7]:
# Data_deterministic

model.Arc = Param(model.I, model.J) #arc available
model.Rev = Param(model.I, model.J) #arc revenue
model.Cost = Param(model.I, model.J) #arc cost
model.B = Param()

In [8]:
#Data_stochastic

model.ArcDemand = Param(model.I, model.J) #arc demand

In [9]:
# Variables

model.X_WS = Param(model.S)
model.X_EV = Param(model.S)
#model.X_RP = Param(initialize=lambda m: model.S[value(model.S)])
model.X = Var(model.S, bounds=(0.0, model.B))
model.Y = Var(model.I, model.J, bounds=(0.0, model.B))
model.Z = Var(model.I, model.J, bounds=(0.0, None))

model.FirstStageProfit = Var()
model.SecondStageProfit = Var()

In [10]:
# Constraints

def vehicle_num_cap_rule(model):
    return sum(model.X[s] for s in model.S) <= model.B
model.VehicleNumCapRule = Constraint(rule=vehicle_num_cap_rule)


def vehicle_assigned_cap_rule(model,s):
    return sum(model.Y[s,j] for j in model.J if model.Arc[s,j]>=1) == model.X[s]
model.RequiredDemandRule = Constraint(model.S, rule=vehicle_assigned_cap_rule)

def flow_balance_rule(model,d):
    return (sum(model.Y[i,d] for i in model.I if model.Arc[i,d]>=1) - sum(model.Y[d,i] for i in model.I if model.Arc[d,i]>=1)) == 0.0
model.FlowBalanceRule = Constraint(model.D, rule=flow_balance_rule)

def overage_rule(model,i,j):
    return model.Y[i,j] - model.ArcDemand[i,j] <= model.Z[i,j]
model.OverageRule = Constraint(model.I, model.J, rule=overage_rule)

def y_rule(model,i,j):
    return (0.0, model.Y[i,j], model.Arc[i,j]*51)
model.YRule = Constraint(model.I, model.J, rule=y_rule)

#NOTE: (Part H) We have added a constraint to fix X at RP
##def x_fix_rule(model,s):
##    return model.X[s] == model.X_RP[s]
##model.XFixRule = Constraint(model.S, rule=x_fix_rule)

In [11]:
# Stage-specific cost computations

def first_stage_profit_rule(model):
    return model.FirstStageProfit == 0.0
model.ComputeFirstStageProfit = Constraint(rule=first_stage_profit_rule)

def second_stage_profit_rule(model):
    return model.SecondStageProfit - sum(sum(model.Rev[i,j] * model.Y[i,j] - (model.Rev[i,j] + model.Cost[i,j])* model.Z[i,j] \
                                           for i in model.I) for j in model.J) == 0.0
model.ComputeSecondStageProfit = Constraint(rule=second_stage_profit_rule)

In [12]:
# Objective

def total_profit_rule(model):
    return (model.FirstStageProfit + model.SecondStageProfit)

model.Total_Profit_Objective = Objective(rule=total_profit_rule, sense=maximize)

In [13]:
# Solve WS for given number of sample realizations with fixed X at X_WS
numSamples=100
numX=5
optVal=numpy.array([0 for i in range(numSamples)])

In [30]:
for i in range(numSamples):
    datafile='svrp_data.dat'
    #datafile = '../drive/MyDrive/Colab_Data/Scenario' + str(i+1) + '.dat'
    instance = model.create_instance(datafile)
    opt= pe.SolverFactory("glpk")
    results = opt.solve(instance, tee=True)
    instance.solutions.store_to(results)
    optVal[i] = value(instance.Total_Profit_Objective)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp_y9gysyr.glpk.raw --wglp /tmp/tmp3c5dorwy.glpk.glp --cpxlp
 /tmp/tmpcegpbx60.pyomo.lp
Reading problem data from '/tmp/tmpcegpbx60.pyomo.lp'...
1223 rows, 808 columns, 1802 non-zeros
6288 lines were read
Writing problem data to '/tmp/tmp3c5dorwy.glpk.glp'...
5447 lines were written
GLPK Simplex Optimizer, v4.65
1223 rows, 808 columns, 1802 non-zeros
Preprocessing...
66 rows, 97 columns, 194 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 66
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (28)
*    50: obj =   1.925580000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.8 Mb (825388 bytes)
Writing basic solution to '/tmp/tmp_y9gysyr.glpk.raw'...
2040 lines were written
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) spec

In [31]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19255.8
  Upper bound: 19255.8
  Number of objectives: 1
  Number of constraints: 1223
  Number of variables: 808
  Number of nonzeros: 1802
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.020672321319580078
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [32]:
instance.pprint()

12 Set Declarations
    ArcDemand_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :  400 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 18), (4, 19), (4, 20), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (

In [33]:
instance.display()

Model unknown

  Variables:
    X : Size=5, Index=S
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :   0.0 :  7.35 :    51 : False : False :  Reals
          2 :   0.0 :  11.7 :    51 : False : False :  Reals
          3 :   0.0 :   5.1 :    51 : False : False :  Reals
          4 :   0.0 :   8.3 :    51 : False : False :  Reals
          5 :   0.0 :  4.95 :    51 : False : False :  Reals
    Y : Size=400, Index=Y_index
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (1, 1) :   0.0 :   0.0 :    51 : False : False :  Reals
          (1, 2) :   0.0 :   0.0 :    51 : False : False :  Reals
          (1, 3) :   0.0 :   0.0 :    51 : False : False :  Reals
          (1, 4) :   0.0 :   0.0 :    51 : False : False :  Reals
          (1, 5) :   0.0 :   0.0 :    51 : False : False :  Reals
          (1, 6) :   0.0 :   3.1 :    51 : False : False :  Reals
          (1, 7) :   0.0 :   0.9 :    51 : False : False :  Reals
          (1, 8) :   0